In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

path = '/home/sej/workspace/dyno_experiments/experiment13/logs/'
def get_float_values(file_name, mode='spark'):
    if mode == 'path':
        pattern = r'\[DynO elapsedTime\] (\d+)'
    elif mode == 'inf':
        pattern = r'(\d+\.\d+)'
    elif mode == 'total':
        pattern = r'finished: start at Main\.scala:\d+, took (\d+\.\d+)'
    elif mode == 'req':
        pattern = r'\[DynO requestTime\] (\d+)'
    elif mode == 'find':
        pattern = r'\[DynO findPathTime\] (\d+)'

    # Open the text file for reading
    with open(file_name, 'r') as file:
        text = file.read()

    # Use re.finditer to find all occurrences of the pattern
    matches = re.finditer(pattern, text)
    float_list = []
    # Extract and print all the float values found
    for match in matches:
        if mode == 'path' or mode == 'req' or mode == 'find':
            float_value = float(match.group(1))
        if mode == 'inf':
            float_value = float(match.group(1))
            float_value *= 1000
        if mode == 'total':
            float_value = float(match.group(1))
            float_value *= 1000
            #print(float_value)
        if float_value > 1000.0:
            float_value = 1000.0
        float_list.append(float_value)
    
    return float_list
 
req_r = []
breq_r = []
def make_cumulative_bar():
    querys=["query2_", "query3_", "query1_"]
    x_query = ["10", "50", "100"]
    x_query2 = ["10", "100", "1000"]
    types=["10_10", "50_50", "100_100"]
    #types2=["10_10", "1000_1000", "100_100"]
    types2=["10_10"]
    labels=['ML inference latency', 'Mapping latency','Communicate latency', 'Path Find latency']
    total = 300000
    colors=sns.color_palette("terrain")
    for query in querys:
        if query == "query2_":
            fig, ax = plt.subplots()
            yerr_inf = []
            yerr_path = []
            yerr_req = []
            yerr_find = []
            inf_r = []
            path_r = []
            req_r = []
            find_r = []
            
            bpath_r = []
            breq_r = []
            bfind_r = []
            
            for type in types2:
                spark_log = path+query+"dyno_"+type+"_5m.txt"
                time_log = path+query+"time_elapsed_"+type+".txt"
                baseline_log = path+query+"baseline_"+type+"_5m.txt"
                
                l_pathfind_latency = get_float_values(spark_log, mode='path')
                pathfind_latency = sum(l_pathfind_latency)/len(l_pathfind_latency)

                l_inference_latency = get_float_values(time_log, mode='inf')
                mapping_latency = sum(l_inference_latency)/len(l_inference_latency)
                
                l_req_latency = get_float_values(spark_log, mode='req')
                req_latency = sum(l_req_latency)/len(l_req_latency)
                
                l_find_latency = get_float_values(spark_log, mode='find')
                find_latency = sum(l_find_latency)/len(l_find_latency)
                
                ### baseline ###
                bl_pathfind_latency = get_float_values(baseline_log, mode='path')
                bpathfind_latency = sum(bl_pathfind_latency)/len(bl_pathfind_latency)
                
                bl_req_latency = get_float_values(baseline_log, mode='req')
                breq_latency = sum(bl_req_latency)/len(bl_req_latency)
                
                bl_find_latency = get_float_values(baseline_log, mode='find')
                bfind_latency = sum(bl_find_latency)/len(bl_find_latency)
                
                #print(inference_latency)
                inf_r.append(mapping_latency)
                path_r.append(pathfind_latency)
                req_r.append(req_latency)
                find_r.append(find_latency)
                
                ### baseline ###
                bpath_r.append(bpathfind_latency)
                breq_r.append(breq_latency)
                bfind_r.append(bfind_latency)
                
                #print(l_inference_latency)
                #print(total/len(l_inference_latency))
                yerr_inf.append(0.1*np.std(np.array(l_inference_latency)))
                yerr_path.append(0.1*np.std(np.array(l_pathfind_latency)))
                yerr_req.append(0.1*np.std(np.array(l_req_latency)))
                yerr_find.append(0.1*np.std(np.array(l_find_latency)))
                
                return l_req_latency, bl_req_latency
                
            plt.stackplot(x_query, inf_r, path_r, req_r, find_r, labels=labels, colors=colors)
            plt.title("Turn-based Update", fontsize=14, fontweight='bold')
            #plt.errorbar(types, inf_r, yerr=yerr_inf, label='ML inference latency')
            #plt.errorbar(types, path_r, yerr=yerr_path, label='Mapping latency')
            #plt.errorbar(types, req_r, yerr=yerr_req, label='Request latency')
            #plt.errorbar(types, find_r, yerr=yerr_find, label='Path latency')
            plt.legend(loc='upper center', bbox_to_anchor=(0.5,1.15), fancybox=True, ncol=2)
            plt.xlabel("Number of Input Rows",fontsize=14, fontweight='bold')
            plt.ylabel("time (ms)",fontsize=14, fontweight='bold')
            plt.ylim(17,20)
            plt.show()
            
            plt.stackplot(x_query, inf_r, bpath_r, breq_r, bfind_r, labels=labels, colors=colors)
            plt.title("Baseline", fontsize=14, fontweight='bold')
            #plt.errorbar(types, inf_r, yerr=yerr_inf, label='ML inference latency')
            #plt.errorbar(types, path_r, yerr=yerr_path, label='Mapping latency')
            #plt.errorbar(types, req_r, yerr=yerr_req, label='Request latency')
            #plt.errorbar(types, find_r, yerr=yerr_find, label='Path latency')
            plt.legend(loc='upper center', bbox_to_anchor=(0.5,1.15), fancybox=True, ncol=2)
            plt.xlabel("Number of Input Rows",fontsize=14, fontweight='bold')
            plt.ylabel("time (ms)",fontsize=14, fontweight='bold')
            plt.ylim(17,20)
            plt.show()
            
            return req_r, breq_r
            #fig.savefig("/home/sej/workspace/dyno_experiments/experiment13/imgs/"+query+type+".pdf", format='pdf', bbox_inches='tight')
        else:
            fig, ax = plt.subplots()
            yerr_inf = []
            yerr_path = []
            yerr_req = []
            yerr_find = []
            inf_r = []
            path_r = []
            req_r = []
            find_r = []
            for type in types2:
                spark_log = path+query+"dyno_"+type+"_5m.txt"
                time_log = path+query+"time_elapsed_"+type+".txt"
                l_pathfind_latency = get_float_values(spark_log, mode='path')
                pathfind_latency = sum(l_pathfind_latency)/len(l_pathfind_latency)

                l_inference_latency = get_float_values(time_log, mode='inf')
                mapping_latency = sum(l_inference_latency)/len(l_inference_latency)
                
                l_req_latency = get_float_values(spark_log, mode='req')
                req_latency = sum(l_req_latency)/len(l_req_latency)
                
                l_find_latency = get_float_values(spark_log, mode='find')
                find_latency = sum(l_find_latency)/len(l_find_latency)
                
                #print(inference_latency)
                inf_r.append(mapping_latency)
                path_r.append(pathfind_latency)
                req_r.append(req_latency)
                find_r.append(find_latency)
                #print(l_inference_latency)
                #print(total/len(l_inference_latency))
                yerr_inf.append(0.1*np.std(np.array(l_inference_latency)))
                yerr_path.append(0.1*np.std(np.array(l_pathfind_latency)))
                yerr_req.append(0.1*np.std(np.array(l_req_latency)))
                yerr_find.append(0.1*np.std(np.array(l_find_latency)))
                
            plt.stackplot(x_query2, inf_r, path_r, req_r, find_r, labels=labels, colors=colors)
            #plt.errorbar(types, inf_r, yerr=yerr_inf, label='ML inference latency')
            #plt.errorbar(types, path_r, yerr=yerr_path, label='Mapping latency')
            #plt.errorbar(types, req_r, yerr=yerr_req, label='Request latency')
            #plt.errorbar(types, find_r, yerr=yerr_find, label='Path latency')
            legend_prop = {'weight':'bold', 'size':11}
            plt.legend(loc='upper center', bbox_to_anchor=(0.5,1.15), fancybox=True, ncol=2, prop=legend_prop)
            plt.xlabel("Number of Input Rows",fontsize=14, fontweight='bold')
            plt.ylabel("Percentage of average total processing time",fontsize=14, fontweight='bold')
            plt.ylim(45,60)
            plt.show()
            fig.savefig("/home/sej/workspace/dyno_experiments/experiment13/imgs/"+query+type+".pdf", format='pdf', bbox_inches='tight')

req_r, breq_r = make_cumulative_bar()
        

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

path = '/home/sej/workspace/dyno_experiments/experiment13/logs/'
def get_float_values(file_name, mode='spark'):
    if mode == 'path':
        pattern = r'\[DynO elapsedTime\] (\d+)'
    elif mode == 'inf':
        pattern = r'(\d+\.\d+)'
    elif mode == 'total':
        pattern = r'finished: start at Main\.scala:\d+, took (\d+\.\d+)'
    elif mode == 'req':
        pattern = r'\[DynO requestTime\] (\d+)'
    elif mode == 'find':
        pattern = r'\[DynO findPathTime\] (\d+)'

    # Open the text file for reading
    with open(file_name, 'r') as file:
        text = file.read()

    # Use re.finditer to find all occurrences of the pattern
    matches = re.finditer(pattern, text)
    float_list = []
    # Extract and print all the float values found
    for match in matches:
        if mode == 'path' or mode == 'req' or mode == 'find':
            float_value = float(match.group(1))
        if mode == 'inf':
            float_value = float(match.group(1))
            float_value *= 1000
        if mode == 'total':
            float_value = float(match.group(1))
            float_value *= 1000
            #print(float_value)
        if float_value > 1000.0:
            float_value = 1000.0
        float_list.append(float_value)
    
    return float_list
 

def make_cumulative_bar():
    querys=["query2_", "query3_", "query1_"]
    x_query = ["10", "50", "100"]
    x_query2 = ["10", "100", "1000"]
    types=["10_10", "50_50", "100_100"]
    types2=["10_10", "1000_1000", "100_100"]
    labels=['Mapping latency','Communicate latency', 'Path Find latency']
    colors=sns.color_palette("terrain")
    for query in querys:
        if query == "query1_":
            fig, ax = plt.subplots()
            yerr_path = []
            yerr_req = []
            yerr_find = []
            path_r = []
            req_r = []
            find_r = []
            for type in types:
                spark_log = path+query+"dyno_"+type+"_5m.txt"
                time_log = path+query+"time_elapsed_"+type+".txt"
                l_pathfind_latency = get_float_values(spark_log, mode='path')
                pathfind_latency = sum(l_pathfind_latency)/len(l_pathfind_latency)
                
                l_req_latency = get_float_values(spark_log, mode='req')
                req_latency = sum(l_req_latency)/len(l_req_latency)
                
                l_find_latency = get_float_values(spark_log, mode='find')
                find_latency = sum(l_find_latency)/len(l_find_latency)
                
                path_r.append(pathfind_latency)
                req_r.append(req_latency)
                find_r.append(find_latency)

                yerr_path.append(0.1*np.std(np.array(l_pathfind_latency)))
                yerr_req.append(0.1*np.std(np.array(l_req_latency)))
                yerr_find.append(0.1*np.std(np.array(l_find_latency)))
                
            plt.stackplot(x_query, path_r, req_r, find_r, labels=labels, colors=colors)
            legend_prop = {'weight':'bold', 'size':11}
            plt.legend(loc='upper center', bbox_to_anchor=(0.5,1.15), fancybox=True, ncol=2, prop=legend_prop)
            plt.xlabel("Number of Input Rows",fontsize=14, fontweight='bold')
            plt.ylabel("Latency (ms)",fontsize=14, fontweight='bold')
            plt.ylim(0, 10)
            plt.show()
            fig.savefig("/home/sej/workspace/dyno_experiments/experiment13/imgs/"+query+type+".pdf", format='pdf', bbox_inches='tight')
        else:
            fig, ax = plt.subplots()
            yerr_path = []
            yerr_req = []
            yerr_find = []
            path_r = []
            req_r = []
            find_r = []
            for type in types2:
                spark_log = path+query+"dyno_"+type+"_5m.txt"
                time_log = path+query+"time_elapsed_"+type+".txt"
                l_pathfind_latency = get_float_values(spark_log, mode='path')
                pathfind_latency = sum(l_pathfind_latency)/len(l_pathfind_latency)
                
                l_req_latency = get_float_values(spark_log, mode='req')
                req_latency = sum(l_req_latency)/len(l_req_latency)
                
                l_find_latency = get_float_values(spark_log, mode='find')
                find_latency = sum(l_find_latency)/len(l_find_latency)

                path_r.append(pathfind_latency)
                req_r.append(req_latency)
                find_r.append(find_latency)

                yerr_path.append(0.1*np.std(np.array(l_pathfind_latency)))
                yerr_req.append(0.1*np.std(np.array(l_req_latency)))
                yerr_find.append(0.1*np.std(np.array(l_find_latency)))
                
            plt.stackplot(x_query2, path_r, req_r, find_r, labels=labels, colors=colors)

            legend_prop = {'weight':'bold', 'size':11}
            plt.legend(loc='upper center', bbox_to_anchor=(0.5,1.15), fancybox=True, ncol=2, prop=legend_prop)
            plt.xlabel("Number of Input Rows",fontsize=14, fontweight='bold')
            plt.ylabel("Latency (ms)",fontsize=14, fontweight='bold')
            plt.ylim(0, 2)
            plt.show()
            fig.savefig("/home/sej/workspace/dyno_experiments/experiment13/imgs/"+query+type+".pdf", format='pdf', bbox_inches='tight')

make_cumulative_bar()
        

In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# X axis : Q1, Q2, Q3
# Y axis : Additional Latency (ms)
# cumulative bar plot
path = '/home/sej/workspace/dyno_experiments/experiment13/logs/'
def get_float_values(file_name, mode='spark'):
    if mode == 'path':
        pattern = r'\[DynO elapsedTime\] (\d+)'
    elif mode == 'inf':
        pattern = r'(\d+\.\d+)'
    elif mode == 'total':
        pattern = r'finished: start at Main\.scala:\d+, took (\d+\.\d+)'
    elif mode == 'req':
        pattern = r'\[DynO requestTime\] (\d+)'
    elif mode == 'find':
        pattern = r'\[DynO findPathTime\] (\d+)'

    # Open the text file for reading
    with open(file_name, 'r') as file:
        text = file.read()

    # Use re.finditer to find all occurrences of the pattern
    matches = re.finditer(pattern, text)
    float_list = []
    # Extract and print all the float values found
    for match in matches:
        if mode == 'path' or mode == 'req' or mode == 'find':
            float_value = float(match.group(1))
        if mode == 'inf':
            float_value = float(match.group(1))
            float_value *= 1000
        if mode == 'total':
            float_value = float(match.group(1))
            float_value *= 1000
            #print(float_value)
        if float_value > 1000.0:
            float_value = 1000.0
        float_list.append(float_value)
    
    return float_list
 
def data_preprocessor(query, types):
    path_r = []
    req_r = []
    find_r = []
    
    for type in types:
        spark_log = path+query+"dyno_"+type+"_5m.txt"
        time_log = path+query+"time_elapsed_"+type+".txt"
        l_pathfind_latency = get_float_values(spark_log, mode='path')
        pathfind_latency = sum(l_pathfind_latency)/len(l_pathfind_latency)
        
        l_req_latency = get_float_values(spark_log, mode='req')
        req_latency = sum(l_req_latency)/len(l_req_latency)
        
        l_find_latency = get_float_values(spark_log, mode='find')
        find_latency = sum(l_find_latency)/len(l_find_latency)
        
        path_r.append(pathfind_latency)
        req_r.append(req_latency)
        find_r.append(find_latency)
    return path_r[0], req_r[0], find_r[0]
                
def cumulative_bar():
    querys=["query1_", "query2_", "query3_"]
    #types=["10_10", "50_50", "100_100"]
    #types2=["10_10", "1000_1000", "100_100"]
    types = ["100_100"]
    types2 = ["100_100"]
    labels=['Mapping latency','Communicate latency', 'Path Find latency']
    species=("Q1", "Q2", "Q3")
    values = {
        labels[0]: [],
        labels[1]: [],
        labels[2]: [],
    }
    colors=sns.color_palette("terrain")
    for query in querys:
        if query == "query1_":
            path_r, req_r, find_r = data_preprocessor(query, types)
        else:
            path_r, req_r, find_r = data_preprocessor(query, types2)
        if query == "query2_":
            find_r = 0.12
        values[labels[0]].append(path_r)
        values[labels[1]].append(req_r)
        values[labels[2]].append(find_r)
    
    for i in range(3):
        values[labels[i]] = np.array(values[labels[i]])
    
    width = 0.5
    fig, ax = plt.subplots()
    bottom = np.zeros(3)
    colors=sns.color_palette("terrain")
    i = 0
    for boolean, value in values.items():
        p = ax.bar(species, value, width, label=boolean, bottom=bottom, color=colors[i])
        bottom += value
        i += 1
        ax.bar_label(p, label_type='center',fmt='{:.2f} ms',fontweight='bold')
    #plt.stackplot(x_query, path_r, req_r, find_r, labels=labels, colors=colors)
    legend_prop = {'weight':'bold', 'size':9}
    ax.legend(fancybox=True, ncol=1, prop=legend_prop)
    plt.xlabel("Queries",fontsize=14, fontweight='bold')
    plt.ylabel("Latency (ms)",fontsize=14, fontweight='bold')
    plt.show()
    fig.savefig("/home/sej/workspace/dyno_experiments/experiment13/imgs/full_latency.pdf", format='pdf', bbox_inches='tight')

cumulative_bar()